# Particle Swarm Optimization Code Documentation

Our implementation of PSO relies on two fundamental abstractions (a _solver_ objet which manipulates a collection of _particles_) which are particularly suited to the Object-Oriented Programing (OOP) paradigm.

We therefore implement two python classes which can be imported from the `PSPSO` module (for Parallel Synchronous Particle Swarm Optimization).

## The `Particle` class

### Attributes
We implement a Particle as having the following __attributes__:

- a `position`: the current position of the particle. Initiated uniformly at random in the hypercube $[\texttt{lower}, \texttt{upper}]^{\texttt{ndim}}$ where `lower`, `upper`, and `ndim` are provided by the user during class instantation. Implemented as a `numpy` array.

- a `velocity`: the current velocity of the particle. Initiated as a random requence of -1s and 1s accross `ndim` dimensions. Implemented as a `numpy` array.

- a `personal_best_position` and associated `personal_best_error` to keep track of a particle best known position and error (essentially the particle's memory). Initiated to the particle's original (random) position with an error of positive infinity.

- `c1`, `c2`, `w`: the cognitive, social, and inertia parameters. User-provided, with defaults 1, 2, and .5 respectively.


### Methods

A Particle has the following __methods__:

- `update_velocity(self, global_best)`: given the global best known position, update the particle's velocity according to the PSO rule.

- `move(self)`: move the particle to a new position using the update rule $position_{t+1} = position_{t} + velocity_{t}$

## The `PSO` class

A `PSO` object is a solver that implements the PSO algorithm to minimize a given function. It is instantiated as follows:

```python
PSO(num_particles, function, n_iter,
    ndim, lower = -10, upper = 10,
    c1 = 1, c2 = 2, w = .5, parallel = False)
```



### Attributes

- `particles`: a `list` of `Particle` objects initiated as described above. There are `num_particles` particles.

- `fitnesses`: an `num_particles`-dimensional array containing the current fitness (aka value) of each of the particles.

- `global_best` and associated `global_best_error`: the global best known position along with the function value at that point.

- `function`: the function to be optimised. Has to be written such that it takes a list of positions and returns an array of fitnesses.

- `n_iter`: the number of iterations to run the PSO algorithm for.

- `parallel`: a `True/False` value indicating whether to run the PSO algorithm in Parallel. If `True`, a `pooler` attribute is also created which contains a `Pool` object from the `multiprocessing` module which pools together all your available CPUs at the exception of one.